In [1]:
######################## START HERE FOR CPGA FULL CODE

In [2]:
######################## STEP 1: DATA PREPARATION ####
### STEP 1 ###########################################
######################################################

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time

torch.manual_seed(42)
np.random.seed(42)

DEMO_MODE = True

if DEMO_MODE:
    excel_path = './data/demo_dataset.xlsx'
    image_dir = './data/images/LAYERS_EXTRACTED_CPGA'
else:
    excel_path = '/path/to/your/full_dataset.xlsx' 
    image_dir = '/path/to/your/full_images'

if os.path.exists(excel_path):
    data = pd.read_excel(excel_path)
    print(f"Loaded dataset from: {excel_path}")
else:
    raise FileNotFoundError(f"Dataset not found at {excel_path}. Please check README.")

geometry_mapping = {
    'Primitive': 'R1',
    'Diamond': 'R2',
    'Gyroid': 'R3',
    'Nevious': 'R4',
    'FRD': 'R5',
    'Fkoch': 'R6'
}
data['geometry_mapped'] = data['geometry'].map(geometry_mapping)

def map_lattice_c(value):
    if value in [0.3, 1.3, 0.8]:
        return 1
    elif value in [0.4, 1.4, 0.9]:
        return 2
    elif value in [0.5, 1.5, 1.0]:
        return 3
    else:
        return None

data['lattice_c_mapped'] = data['lattice_c'].apply(map_lattice_c)

def get_image_paths(lattice_n, lattice_c_mapped, geometry_mapped):
    image_paths = []
    for i in range(20):
        image_name = f"U{lattice_n}C{lattice_c_mapped}{geometry_mapped}_{i:02d}.png"
        image_path = os.path.join(image_dir, image_name)
        if os.path.exists(image_path):
            image_paths.append(image_path)
    return image_paths

data['image_paths'] = data.apply(
    lambda row: get_image_paths(row['lattice_n'], row['lattice_c_mapped'], row['geometry_mapped']),
    axis=1
)
data = data[data['image_paths'].apply(lambda x: len(x) == 20)]

image_transforms = transforms.Compose([
    transforms.Resize((150, 150)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

def load_images(image_paths):
    """Load a stack of 20 grayscale images for one sample."""
    images = []
    for image_path in image_paths:
        image = Image.open(image_path).convert('L')
        transformed_image = image_transforms(image)
        images.append(transformed_image)
    images = torch.stack(images)          
    images = images.permute(1, 0, 2, 3)      
    return images

Loaded dataset from: ./data/demo_dataset.xlsx


In [3]:
######################## STEP 2: DATASET and MODEL ###
### STEP 2 ###########################################
######################################################

class MultimodalDataset(Dataset):
    def __init__(self, numeric_data, image_data, targets):
        self.numeric_data = torch.tensor(numeric_data.values, dtype=torch.float32)
        self.image_data = image_data
        self.targets = torch.tensor(targets.values, dtype=torch.float32)

        DEMO_MODE = True
        
        if DEMO_MODE:
            self.original_image_dir = './data/images/LAYERS_EXTRACTED_CPGA'
            self.convolved_image_dir = './data/images/LAYERS_EXTRACTED_CONV_CPGA'
        else:
            self.original_image_dir = '/path/to/your/original_images.xlsx' 
            self.convolved_image_dir = '/path/to/your/convolved_images'
    
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        numeric_features = self.numeric_data[idx]
        image_paths = self.image_data.iloc[idx]
        
        original_image_paths = [os.path.join(self.original_image_dir, os.path.basename(p)) for p in image_paths]
        convolved_image_paths = [os.path.join(self.convolved_image_dir, os.path.basename(p)) for p in image_paths]

        original_images = load_images(original_image_paths)
        convolved_images = load_images(convolved_image_paths)
        target = self.targets[idx]

        return numeric_features, original_images, convolved_images, target

class MultimodalModel(nn.Module):
    def __init__(self, image_size=150):
        super().__init__()

        self.final_spatial = image_size // (2 ** 4) 

        self.numeric_model = nn.Sequential(
            nn.Linear(6, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU()
        )
        
        num_numeric_features = 32

        self.film1_gamma = nn.Linear(num_numeric_features, 32) 
        self.film1_beta  = nn.Linear(num_numeric_features, 32)
        
        self.film2_gamma = nn.Linear(num_numeric_features, 64) 
        self.film2_beta  = nn.Linear(num_numeric_features, 64)
        
        self.film3_gamma = nn.Linear(num_numeric_features, 128) 
        self.film3_beta  = nn.Linear(num_numeric_features, 128)
        
        self.film4_gamma = nn.Linear(num_numeric_features, 256) 
        self.film4_beta  = nn.Linear(num_numeric_features, 256)
       
        self.orig_enc1 = nn.Sequential(nn.Conv3d(1, 32, 3, 1, 1), nn.InstanceNorm3d(32), nn.ReLU())
        self.orig_enc2 = nn.Sequential(nn.Conv3d(32, 64, 3, 1, 1), nn.InstanceNorm3d(64), nn.ReLU())
        self.orig_enc3 = nn.Sequential(nn.Conv3d(64, 128, 3, 1, 1), nn.InstanceNorm3d(128), nn.ReLU())
        self.orig_enc4 = nn.Sequential(nn.Conv3d(128, 256, 3, 1, 1), nn.InstanceNorm3d(256), nn.ReLU())

        self.conv_enc1 = nn.Sequential(nn.Conv3d(1, 32, 3, 1, 1), nn.InstanceNorm3d(32), nn.ReLU())
        self.conv_enc2 = nn.Sequential(nn.Conv3d(32, 64, 3, 1, 1), nn.InstanceNorm3d(64), nn.ReLU())
        self.conv_enc3 = nn.Sequential(nn.Conv3d(64, 128, 3, 1, 1), nn.InstanceNorm3d(128), nn.ReLU())
        self.conv_enc4 = nn.Sequential(nn.Conv3d(128, 256, 3, 1, 1), nn.InstanceNorm3d(256), nn.ReLU())
        
        self.pool = nn.MaxPool3d(2)

        num_image_channels = 256 + 256
        self.fusion_head = nn.Sequential(
            nn.Conv3d(num_image_channels, 256, kernel_size=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256 * self.final_spatial * self.final_spatial, 128),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(128, 1)
        )


    def apply_film(self, x, gamma, beta):

        gamma = gamma.view(gamma.shape[0], -1, 1, 1, 1)
        beta = beta.view(beta.shape[0], -1, 1, 1, 1)
        return (gamma * x) + beta

    def forward(self, numeric_data, original_image_data, convolved_image_data):

        f_num = self.numeric_model(numeric_data) 

        g1, b1 = self.film1_gamma(f_num), self.film1_beta(f_num)
        g2, b2 = self.film2_gamma(f_num), self.film2_beta(f_num)
        g3, b3 = self.film3_gamma(f_num), self.film3_beta(f_num)
        g4, b4 = self.film4_gamma(f_num), self.film4_beta(f_num)
        
        f_orig = original_image_data
        f_conv = convolved_image_data

        f_orig = self.orig_enc1(f_orig)
        f_conv = self.conv_enc1(f_conv)
        f_orig = self.apply_film(f_orig, g1, b1)
        f_conv = self.apply_film(f_conv, g1, b1)
        f_orig, f_conv = self.pool(f_orig), self.pool(f_conv)

        f_orig = self.orig_enc2(f_orig)
        f_conv = self.conv_enc2(f_conv)
        f_orig = self.apply_film(f_orig, g2, b2)
        f_conv = self.apply_film(f_conv, g2, b2)
        f_orig, f_conv = self.pool(f_orig), self.pool(f_conv)

        f_orig = self.orig_enc3(f_orig)
        f_conv = self.conv_enc3(f_conv)
        f_orig = self.apply_film(f_orig, g3, b3)
        f_conv = self.apply_film(f_conv, g3, b3)
        f_orig, f_conv = self.pool(f_orig), self.pool(f_conv)

        f_orig = self.orig_enc4(f_orig)
        f_conv = self.conv_enc4(f_conv)
        f_orig = self.apply_film(f_orig, g4, b4)
        f_conv = self.apply_film(f_conv, g4, b4)
        f_orig, f_conv = self.pool(f_orig), self.pool(f_conv)

        img_feats = torch.cat([f_orig, f_conv], dim=1)
        output = self.fusion_head(img_feats)
        return output

In [4]:
######################## STEP 3: TRAINING UTILITIES ##
### STEP 3 ###########################################
######################################################

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
import torch
from torch.optim import lr_scheduler
import joblib 

X_numeric = data[['sav_ratio', 'void_ratio', 'lattice_n', 'Layer height', 'Intensity', 'mass_before']]
X_images = data['image_paths']
y = data['degree_of_conversion']

X_numeric = X_numeric.apply(pd.to_numeric, errors='coerce').fillna(0)

X_num_train, X_num_test, X_img_train, X_img_test, y_train, y_test = train_test_split(
    X_numeric, X_images, y, test_size=0.2, random_state=42
)
X_num_train, X_num_val, X_img_train, X_img_val, y_train, y_val = train_test_split(
    X_num_train, X_img_train, y_train, test_size=0.2, random_state=42
)

print("Normalizing numerical features...")
scaler = StandardScaler()
scaler.fit(X_num_train)

joblib.dump(scaler, 'numeric_scaler.pkl')

numeric_cols = X_num_train.columns
def scale_df(df):
    return pd.DataFrame(scaler.transform(df), columns=numeric_cols, index=df.index)

X_num_train, X_num_val, X_num_test = map(scale_df, [X_num_train, X_num_val, X_num_test])
print("Normalization complete.")

y_train = pd.to_numeric(y_train, errors='coerce').fillna(0)
y_val   = pd.to_numeric(y_val, errors='coerce').fillna(0)
y_test  = pd.to_numeric(y_test, errors='coerce').fillna(0)

y_train_log, y_val_log, y_test_log = map(np.log1p, [y_train, y_val, y_test])

train_dataset_log = MultimodalDataset(X_num_train, X_img_train, y_train_log)
val_dataset_log   = MultimodalDataset(X_num_val,   X_img_val,   y_val_log)
test_dataset_log  = MultimodalDataset(X_num_test,  X_img_test,  y_test_log)

train_loader_log = DataLoader(train_dataset_log, batch_size=32, shuffle=True,  num_workers=8, pin_memory=True)
val_loader_log   = DataLoader(val_dataset_log,   batch_size=32, shuffle=False, num_workers=8, pin_memory=True)
test_loader_log  = DataLoader(test_dataset_log,  batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultimodalModel(image_size=150).to(device)

def weighted_mse_loss(output, target):
    weight = torch.ones_like(target)
    weight = torch.where((target >= 0.8) & (target < 0.87), 4, weight)
    weight = torch.where((target >= 0.7) & (target < 0.8), 5, weight)
    weight = torch.where((target >= 0.6) & (target < 0.7), 6, weight)
    weight = torch.where(target <= 0.6, 7, weight)
    return (weight * (output - target) ** 2).mean()

criterion = weighted_mse_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.00015)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
best_val_loss = float('inf')


Normalizing numerical features...
Normalization complete.


In [5]:
######################## STEP 4: RUN TRAINING ########
### STEP 4 ###########################################
######################################################

from torch.cuda.amp import GradScaler, autocast
import numpy as np
import time
import torch

train_losses, val_losses = [], []
best_val_loss = float('inf')
patience_counter = 0
early_stopping_patience = 20  
scaler = GradScaler()          

log_file = open("training_log_CPGA.txt", "a", buffering=1)

print(f"Starting training on {device} with mixed precision...")

for epoch in range(500):

    model.train()
    running_loss = 0.0
    epoch_start = time.time()

    for batch_idx, (numeric_features, original_images, convolved_images, targets) in enumerate(train_loader_log):
        numeric_features = numeric_features.to(device, non_blocking=True)
        original_images = original_images.to(device, non_blocking=True)
        convolved_images = convolved_images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast():
            outputs = model(numeric_features, original_images, convolved_images)
            loss = criterion(outputs.squeeze(), targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader_log)
    train_losses.append(avg_train_loss)
    scheduler.step()

    model.eval()
    val_loss = 0.0
    with torch.no_grad(), autocast():
        for numeric_features, original_images, convolved_images, targets in val_loader_log:
            numeric_features = numeric_features.to(device, non_blocking=True)
            original_images = original_images.to(device, non_blocking=True)
            convolved_images = convolved_images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            outputs = model(numeric_features, original_images, convolved_images)
            loss = criterion(outputs.squeeze(), targets)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader_log)
    val_losses.append(avg_val_loss)
    epoch_time = time.time() - epoch_start

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_val_loss': best_val_loss,
            'train_losses': train_losses,
            'val_losses': val_losses,
        }, 'best_model_CPGA.pth')
    else:
        patience_counter += 1

    if (epoch + 1) % 5 == 0 or epoch == 0:
        msg = (f"Epoch [{epoch+1}/500] | Time: {epoch_time:.1f}s | "
               f"Train Loss: {avg_train_loss:.5f} | Val Loss: {avg_val_loss:.5f} | "
               f"Best: {best_val_loss:.5f}")
        print(msg)
        log_file.write(msg + "\n")

log_file.close()
print("Training complete")

Starting training on cuda with mixed precision...


/tmp/ipykernel_816/823269859.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_816/823269859.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/mnt/d/Ubuntu/miniconda3/envs/torch5090/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
/tmp/ipykernel_816/823269859.py:50: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.aut

Epoch [1/500] | Time: 3.3s | Train Loss: 1.98814 | Val Loss: 1.85232 | Best: 1.85232
Epoch [5/500] | Time: 1.2s | Train Loss: 0.35715 | Val Loss: 0.17510 | Best: 0.17510
Epoch [10/500] | Time: 1.1s | Train Loss: 0.02509 | Val Loss: 0.08858 | Best: 0.01747
Epoch [15/500] | Time: 1.1s | Train Loss: 0.20688 | Val Loss: 0.02483 | Best: 0.01747
Epoch [20/500] | Time: 1.1s | Train Loss: 0.04803 | Val Loss: 0.01233 | Best: 0.01233
Epoch [25/500] | Time: 1.1s | Train Loss: 0.08565 | Val Loss: 0.02441 | Best: 0.00904
Epoch [30/500] | Time: 1.1s | Train Loss: 0.02682 | Val Loss: 0.00597 | Best: 0.00597
Epoch [35/500] | Time: 1.1s | Train Loss: 0.06680 | Val Loss: 0.00585 | Best: 0.00493
Epoch [40/500] | Time: 1.2s | Train Loss: 0.05501 | Val Loss: 0.02823 | Best: 0.00453
Epoch [45/500] | Time: 1.1s | Train Loss: 0.03563 | Val Loss: 0.00758 | Best: 0.00453
Epoch [50/500] | Time: 1.2s | Train Loss: 0.01097 | Val Loss: 0.01669 | Best: 0.00453
Epoch [55/500] | Time: 1.1s | Train Loss: 0.08167 | Val 